In [15]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import DatasetDict, Sequence, Value, Features, Dataset, concatenate_datasets
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
import numpy as np
import os
import sys
sys.path.append(os.getcwd()+"/../..")
from src import paths
from src.utils import model_output
import tqdm

In [2]:
# # Run this cell if you want to download and fine-tune the model
# from huggingface_hub import notebook_login

# # Login to Hugging Face Hub as model is gated
# notebook_login()

# # Checkpoint
# checkpoint = "GerMedBERT/medbert-512"

# # Load tokenizer
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# # Save tokenizer
# tokenizer.save_pretrained(paths.MODEL_PATH/'medbert-512')

# # Load model for embedding
# model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=num_labels, problem_type="multi_label_classification")

# # Save model
# model.save_pretrained(paths.MODEL_PATH/'medbert-512')

In [3]:
# Load dataset
dataset = DatasetDict.load_from_disk(paths.DATA_PATH_PREPROCESSED/'line_labelling/line_labelling_clean_dataset')

# Num Labels
num_labels = len(set(dataset['train']['class_agg']))

In [4]:
# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(paths.MODEL_PATH/'medbert-512')

# Load model
model = AutoModelForSequenceClassification.from_pretrained(paths.MODEL_PATH/'medbert-512', num_labels=num_labels, problem_type="multi_label_classification").to(device)

In [5]:
# Tokenize
def tokenize(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256, return_tensors='pt')

# # Set format of labels to FloatTensor
features = Features({'labels': Sequence(Value(dtype='float32')),
                     'input_ids': Sequence(Value(dtype='int32')),
                     'attention_mask': Sequence(Value(dtype='int32')),
                     'token_type_ids': Sequence(Value(dtype='int32')),
                     'class_agg': Value(dtype='string'),
                     'rid': Value(dtype='string'),
                     'text': Value(dtype='string'),
                     'class': Value(dtype='string')
                     })

# Tokenize dataset
dataset = dataset.map(tokenize, batched=True, features=features)


In [6]:
# Train/Val/Test 
train_dataset = dataset['train']
val_dataset = dataset['val']
test_dataset = dataset['test']

In [7]:
# Training Arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=12,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=200,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    load_best_model_at_end=True,
    save_strategy='epoch',
    evaluation_strategy='epoch',
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    fp16=True,
    seed=42,
)

# Trainer
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset            # evaluation dataset
)

In [8]:
#trainer.train()

In [9]:
# Save model
#trainer.save_model(paths.MODEL_PATH/'medbert-512-finetuned')

In [10]:
# Load model
trainer.model = AutoModelForSequenceClassification.from_pretrained(paths.MODEL_PATH/'medbert-512-finetuned', num_labels=num_labels, problem_type="multi_label_classification").to(device)

In [13]:
train_out = model_output(train_dataset, model = trainer.model)
val_out = model_output(val_dataset, model = trainer.model)
test_out = model_output(test_dataset, model = trainer.model)

# Save embeddings
torch.save(train_out, paths.RESULTS_PATH/'line_labelling/medBERT-finetuned-train_output.pt')
torch.save(val_out, paths.RESULTS_PATH/'line_labelling/medBERT-finetuned-val_output.pt')
torch.save(test_out, paths.RESULTS_PATH/'line_labelling/medBERT-finetuned-test_output.pt')

100%|██████████| 11/11 [00:05<00:00,  1.93it/s]
